In [ ]:
import logging
logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

from datasets_metadata import ts_metadata
import pandas as pd

from utilsforecast.plotting import plot_series

from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mse, mae, rmse, mape

In [ ]:
dataset_name = "ETTh1"
exog_list = ts_metadata[dataset_name]["exog_list"]
target_ts = ts_metadata[dataset_name]["target_ts"]
freq = ts_metadata[dataset_name]["freq"]

df = pd.read_csv(f"./processed_data/{dataset_name}.csv")
df["ds"] = pd.to_datetime(df["ds"])
df

In [ ]:
from models import get_nf

# nf = NeuralForecast.load(path=f'./saved_models/{dataset_name}')

horizon = 24

nf = get_nf(
    horizon=horizon,
    freq=freq,
    exog_list=exog_list,
    num_samples=20,
    backend="optuna"
)

# cv_df = nf.cross_validation(df, n_windows=10, step_size=100)

cv_df = nf.cross_validation(df, val_size=1000, test_size=2000, n_windows=None)

In [ ]:
cv_df.columns = cv_df.columns.str.replace('-median', '')
cv_df

In [ ]:
# unique_id_filter = "OT"
unique_id_filter = "OT_exogenous_FARM_shaped"

filtered_cv_df = cv_df[cv_df["unique_id"] == unique_id_filter]
filtered_df = df[df["unique_id"] == unique_id_filter]

filtered_cv_df = filtered_cv_df.reset_index(drop=True)
filtered_df = filtered_df.reset_index(drop=True)

cutoff_index=0

total_cutoffs = len(filtered_cv_df["cutoff"].unique())
cutoff_value = filtered_cv_df["cutoff"].unique()[cutoff_index]
cutoff_time_index = filtered_df.index[filtered_df["ds"] == cutoff_value].tolist()
if len(cutoff_time_index) > 1:
    raise Exception(f"Unexpected cutoff_time_index length value: {len(cutoff_time_index)}. It should be always 1.")

cutoff_time_index = cutoff_time_index[0]
lookback = 120
horizon = 24

plot_series(
    filtered_df[cutoff_time_index-120:cutoff_time_index+horizon+1],
    filtered_cv_df[filtered_cv_df["cutoff"] == filtered_cv_df["cutoff"].unique()[cutoff_index]].drop(columns=['cutoff', 'y']),
    # cv_df.drop(columns="cutoff"),
    # max_insample_length= 12 * 10,
    ids=[unique_id_filter],
    engine="plotly"
)

In [ ]:
evaluation_df = evaluate(cv_df.drop(columns='cutoff'), metrics=[mse, mae, rmse, mape])
evaluation_df['best_model'] = evaluation_df.drop(columns=['metric', 'unique_id']).idxmin(axis=1)
evaluation_df.to_csv(f"./results/{dataset_name}")
evaluation_df

In [ ]:
summary_df = evaluation_df.groupby(['metric', 'best_model']).size().sort_values().to_frame()
summary_df = summary_df.reset_index()
summary_df.columns = ['metric', 'model', 'nr. of unique_ids']
summary_df

In [ ]:
nf.save( 
    path=f"./saved_models/{dataset_name}",
    model_index=None, 
    overwrite=True,
    save_dataset=True
)